<a href="https://colab.research.google.com/github/carive47/Finances/blob/main/metricas_visualizaciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade google-auth google-auth-oauthlib google-auth-httplib2 googleapiclient xlsxwriter unidecode pydrive plotly scipy ace_tools
!pip install unidecode
!pip install unidecode pydrive
!pip install xlsxwriter
!plotly scipy

ERROR: Could not find a version that satisfies the requirement googleapiclient (from versions: none)
ERROR: No matching distribution found for googleapiclient
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.4/169.4 kB 2.8 MB/s eta 0:00:00
/bin/bash: line 1: plotly: command not found


In [ ]:
# # Autenticación y Montaje de Google Drive
# %%
from google.colab import drive
from google.colab import auth
from googleapiclient.discovery import build
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from oauth2client.client import GoogleCredentials

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# # Importaciones necesarias

import pandas as pd
import requests
from difflib import get_close_matches
import datetime
import re
import os
import unidecode
import logging
import numpy as np
import uuid
from scipy.stats import zscore
from pathlib import Path
from IPython.display import display
import plotly.express as px
import plotly.figure_factory as ff
import plotly.io as pio

pio.renderers.default = "colab"

In [ ]:
# # Configuración y Funciones de Ayuda
# 1) Ruta al archivo en Drive
excel_file_path = '/content/drive/My Drive/finca_cafe/db_cafe_primitivo_V3_5.xlsx'

# 2. Función robusta para convertir texto-a-número (y manejar Kilos)
def to_money(val):
    if pd.isna(val): return np.nan
    if isinstance(val, (int, float)): return float(val)
    s = re.sub(r"[^\d.,-]", "", str(val))
    if "." in s and "," in s and s.rfind(",") > s.rfind("."):
        s = s.replace(".", "").replace(",", ".")
    elif "," in s and "." not in s:
        s = s.replace(",", ".")
    try: return float(s)
    except: return np.nan

# 3) Leer todas las hojas
xls = pd.ExcelFile(excel_file_path)
sheets = {sheet: xls.parse(sheet) for sheet in xls.sheet_names}


In [ ]:
# 4) Consolidar datos
records = []
for hoja, df_s in sheets.items():
    if 'fecha' not in df_s.columns: continue
    for _, r in df_s.iterrows():
        fecha = pd.to_datetime(r['fecha'], errors='coerce')
        if pd.isna(fecha): continue
        cluster = r.get('cluster_levenshtein') or r.get('cluster_levenstein') or r.get('lote')
        records.append({
            'fecha': fecha,
            'finca': r.get('finca'),
            'cluster': cluster,
            'actividad': hoja,
            'kilos_cafe': to_money(r.get('kiloscafe')),
            'costo_total': to_money(r.get('valortotal') or r.get('valor_n_jornales') or r.get('valor_n_dejornales')),
            'ingreso': to_money(r.get('valor_a_kilos') or r.get('ingreso'))
        })
df = pd.DataFrame(records)

# 5) Limpieza y tiempo
df['mes'] = df['fecha'].dt.to_period('M').dt.to_timestamp()
df['anio'] = df['fecha'].dt.year
# Outliers 99%
for col in ['costo_total','ingreso']:
    cap = df[col].quantile(0.99)
    df.loc[df[col] > cap, col] = np.nan
df[['kilos_cafe','costo_total','ingreso']] = df[['kilos_cafe','costo_total','ingreso']].fillna(0)
df = df.sort_values(['finca','fecha']).reset_index(drop=True)

# 6) Simular área por lote (5–20 ha)
np.random.seed(42)
clusters = df['cluster'].unique()
area_dict = {cl: np.random.uniform(5,20) for cl in clusters}
df['area_ha'] = df['cluster'].map(area_dict)
df['costo_ha'] = df['costo_total'] / df['area_ha']

# 7) Métricas intermedias
# 7.1 Costo total mensual
cost_monthly = df.groupby(['finca','mes'])['costo_total'].sum().reset_index()

# 7.2 Costo acumulado año calendario
df['costo_ac_year'] = df.groupby(['finca','anio'])['costo_total'].cumsum()

# 7.3 Rolling eficiencia (sólo recolección)
df_reco = df[df['actividad']=='recoleccion']
def rolling_eff(df_in, window):
    out = (df_in.set_index('fecha')
                 .groupby('finca')[['costo_total','kilos_cafe']]
                 .rolling(window, min_periods=1)
                 .sum()
                 .reset_index())
    out[f'costo_kg_{window}'] = out['costo_total']/out['kilos_cafe']
    return out

roll_30_reco = rolling_eff(df_reco, '30D')
roll_90_reco = rolling_eff(df_reco, '90D')

# 7.4 Costo por hectárea por lote
cost_ha = df.groupby('cluster')['costo_total'].sum().reset_index()
cost_ha['costo_ha_total'] = cost_ha['costo_total'] / cost_ha['cluster'].map(area_dict)

# 7.5 Costo por actividad (total y %)
act_cost = df.groupby('actividad')['costo_total'].sum().reset_index()
act_cost['pct'] = act_cost['costo_total']/act_cost['costo_total'].sum()

# 7.6 Gantt data
gantt = df.groupby(['cluster','actividad']) \
          .agg(start=('fecha','min'), end=('fecha','max')) \
          .reset_index()

# 7.7 Costo por cosecha (si existe)
if 'cosecha' in df.columns:
    cost_cosecha = df.groupby('cosecha')['costo_total'].sum().reset_index()

# 7.8 Heatmaps
heat_cop_total = df.groupby(['actividad','cluster'])['costo_total'].sum().unstack(fill_value=0)
heat_cop_ha = df.groupby(['actividad','cluster'])['costo_ha'].sum().unstack(fill_value=0)

# Validaciones
print("Costos mensuales"); display(cost_monthly.head())
print("Costos por actividad (%)"); display(act_cost.head())
print("Simulación áreas (ha)"); display(pd.DataFrame.from_dict(area_dict, orient='index', columns=['area_ha']).head())


Costos mensuales


,finca,mes,costo_total
0,la_paloma,2024-01-01,19837000.0
1,la_paloma,2024-02-01,52038200.0
2,la_paloma,2024-03-01,11765100.0
3,la_paloma,2024-04-01,12709600.0
4,la_paloma,2024-05-01,17024600.0


Costos por actividad (%)


,actividad,costo_total,pct
0,actividad_general,160450323.0,0.166679
1,control_maleza,64223640.0,0.066717
2,control_roya,34910000.0,0.036265
3,fertilizacion,92213015.0,0.095793
4,recoleccion,610726200.0,0.634436


Simulación áreas (ha)


,area_ha
NaN,10.618102
None,19.260715
la_paloma_11,15.979909
la_paloma_8,13.979877
la_paloma_0,7.340280


In [ ]:

# --------- Gráficas MVP -------------

# 1) Costo total mensual
fig1 = px.bar(cost_monthly, x='mes', y='costo_total', color='finca',
              title='Costo total mensual por finca',
              labels={'costo_total':'COP','mes':'Mes'})
fig1.show()

# 2) Costo acumulado año calendario
fig2 = px.line(df, x='fecha', y='costo_ac_year', color='finca',
               title='Costo acumulado año calendario',
               labels={'costo_ac_year':'COP acumulados','fecha':'Fecha'})
fig2.show()

# 3a) Eficiencia de recolección (rolling COP/kg)
fig3a = px.line(roll_30_reco, x='fecha', y='costo_kg_30D', color='finca',
                title='Costo/kg (rolling 30d) – recolección',
                labels={'costo_kg_30D':'COP/kg'})
fig3a.show()
fig3b = px.line(roll_90_reco, x='fecha', y='costo_kg_90D', color='finca',
                title='Costo/kg (rolling 90d) – recolección',
                labels={'costo_kg_90D':'COP/kg'})
fig3b.show()

# 3b) Costo por hectárea
fig4 = px.bar(cost_ha, x='cluster', y='costo_ha_total',
              title='Costo total por hectárea (COP/ha) por lote',
              labels={'costo_ha_total':'COP/ha','cluster':'Lote'})
fig4.show()

# 4) Costo por tipo de actividad
fig5 = px.bar(act_cost, x='actividad', y='costo_total',
              title='Costo total por actividad',
              labels={'costo_total':'COP','actividad':'Actividad'})
fig5.show()

# 5) Distribución porcentual de costos por actividad
fig6 = px.pie(act_cost, names='actividad', values='pct', hole=0.4,
              title='% costos por actividad')
fig6.update_traces(texttemplate='%{label}: %{percent:.1%}')
fig6.show()

# 6) Línea de tiempo de labores (Gantt)
fig7 = px.timeline(
    gantt,
    x_start='start', x_end='end',
    y='cluster', color='actividad',
    title='Cronograma de labores por lote'
)
# invierte el eje para que la primera fila esté arriba
fig7.update_yaxes(autorange='reversed')
fig7.show()


# 7) Comparativo de costos por cosecha
if 'cost_cosecha' in locals():
    fig8 = px.bar(cost_cosecha, x='cosecha', y='costo_total',
                  title='Costo total por tipo de cosecha',
                  labels={'costo_total':'COP','cosecha':'Cosecha'})
    fig8.show()

# 8a) Heatmap COP total
fig9 = px.imshow(heat_cop_total, aspect='auto', color_continuous_scale='Blues',
                 title='Heatmap – Costo total por actividad y lote',
                 labels={'color':'COP','x':'Lote','y':'Actividad'})
fig9.show()

# 8b) Heatmap COP/ha
fig10 = px.imshow(heat_cop_ha, aspect='auto', color_continuous_scale='Viridis',
                  title='Heatmap – Costo/ha por actividad y lote',
                  labels={'color':'COP/ha','x':'Lote','y':'Actividad'})
fig10.show()

#9) boxplot Actividad
df_eff = df[(df['kilos_cafe'] > 0) & (df['costo_total'] > 0)].copy()
df_eff['costo_por_kg'] = df_eff['costo_total'] / df_eff['kilos_cafe']

if df_eff.empty:
    print("⚠️  No hay registros con costo y kilos > 0")
else:
    fig6 = px.box(
        df_eff, y='finca', x='costo_por_kg', points='all', boxmode='overlay',
        color='finca',
        labels={'costo_por_kg':'Costo COP/kg','finca':'Finca'},
        title='Distribución de costo por kilo — Box-plot'
    )
    fig6.show()


# gastos acumulados
df = df.sort_values(['finca','fecha'])
df['year'] = df['fecha'].dt.year
# Acumulados por finca y año
df['costo_ac_year'] = df.groupby(['finca','year'])['costo_total'].cumsum()
df['kilos_ac_year'] = df.groupby(['finca','year'])['kilos_cafe'].cumsum()
df['costo_kg_ac_year'] = df['costo_ac_year'] / df['kilos_ac_year']

fig4 = px.line(
    df, x='fecha', y='costo_kg_ac_year', color='finca',
    title='Costo/kg acumulado año calendario',
    labels={'costo_kg_ac_year':'COP/kg','fecha':'Fecha'}
)
fig4.update_yaxes(tickprefix='$')
fig4.show()




In [ ]:
import plotly.io as pio

# Lista para almacenar los HTML de cada figura
html_figures = []

# Generar HTML para cada figura y agregarlo a la lista
html_figures.append(pio.to_html(fig1, full_html=False))
html_figures.append(pio.to_html(fig2, full_html=False))
html_figures.append(pio.to_html(fig3a, full_html=False))
html_figures.append(pio.to_html(fig3b, full_html=False))
html_figures.append(pio.to_html(fig4, full_html=False))
html_figures.append(pio.to_html(fig5, full_html=False))
html_figures.append(pio.to_html(fig6, full_html=False))
html_figures.append(pio.to_html(fig7, full_html=False))

# Agregar costo por cosecha si existe
if 'cost_cosecha' in locals():
    html_figures.append(pio.to_html(fig8, full_html=False))

html_figures.append(pio.to_html(fig9, full_html=False))
html_figures.append(pio.to_html(fig10, full_html=False))

# Agregar boxplot si fue generado
if 'fig6' in locals():
     html_figures.append(pio.to_html(fig6, full_html=False))

html_figures.append(pio.to_html(fig4, full_html=False))


# Combinar todos los HTML en un solo archivo
all_figures_html = "<html><head><title>Reporte de Gráficas</title></head><body>"
all_figures_html += "<h1>Reporte de Gráficas de Finca de Café</h1>"

for html in html_figures:
    all_figures_html += html
    all_figures_html += "<hr>" # Añade una línea separadora entre gráficas

all_figures_html += "</body></html>"

# Guardar el archivo HTML combinado
with open("reporte_graficas.html", "w") as f:
    f.write(all_figures_html)

print("Archivo 'reporte_graficas.html' generado exitosamente.")


/content/drive/MyDrive/finca_cafe/db_cafe_primitivo_V3_5.xlsx

Archivo 'reporte_graficas.html' generado exitosamente.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import re
import numpy as np
import pandas as pd
from pathlib import Path
from IPython.display import display
import plotly.express as px
import plotly.figure_factory as ff
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# 1) Ruta al archivo en Drive
excel_file_path = '/content/drive/MyDrive/finca_cafe/db_cafe_primitivo_V3_5.xlsx'

# 2) Función para convertir textos a número
def to_money(val):
    if pd.isna(val): return np.nan
    if isinstance(val, (int, float)): return float(val)
    s = re.sub(r"[^\d.,-]", "", str(val))
    if "." in s and "," in s and s.rfind(",") > s.rfind("."):
        s = s.replace(".", "").replace(",", ".")
    elif "," in s and "." not in s:
        s = s.replace(",", ".")
    try:
        return float(s)
    except:
        return np.nan

# 3) Leer todas las hojas del Excel
xls = pd.ExcelFile(excel_file_path)
sheets = {sheet: xls.parse(sheet) for sheet in xls.sheet_names}

# 4) Consolidar datos en un único DataFrame
records = []
for hoja, df_s in sheets.items():
    if 'fecha' not in df_s.columns:
        continue
    for _, r in df_s.iterrows():
        fecha = pd.to_datetime(r['fecha'], errors='coerce')
        if pd.isna(fecha):
            continue
        cluster = r.get('cluster_levenshtein') or r.get('cluster_levenstein') or r.get('lote')
        records.append({
            'fecha': fecha,
            'finca': r.get('finca'),
            'cluster': cluster,
            'actividad': hoja,
            'kilos_cafe': to_money(r.get('kiloscafe')),
            'costo_total': to_money(
                r.get('valortotal') or
                r.get('valor_n_jornales') or
                r.get('valor_n_dejornales')
            ),
            'ingreso': to_money(r.get('valor_a_kilos') or r.get('ingreso'))
        })
df = pd.DataFrame(records)

# 5) Limpieza y columnas de tiempo
df['mes'] = df['fecha'].dt.to_period('M').dt.to_timestamp()
df['anio'] = df['fecha'].dt.year

# 5.1) Capa outliers extremos (percentil 99) en costo_total e ingreso
for col in ['costo_total', 'ingreso']:
    cap = df[col].quantile(0.99)
    df.loc[df[col] > cap, col] = np.nan

# 5.2) Rellenar NaN con 0
df[['kilos_cafe', 'costo_total', 'ingreso']] = df[['kilos_cafe', 'costo_total', 'ingreso']].fillna(0)
df = df.sort_values(['finca', 'fecha']).reset_index(drop=True)

# 6) Simular área por lote (5–20 ha)
np.random.seed(42)
clusters = df['cluster'].unique()
area_dict = {cl: np.random.uniform(5, 20) for cl in clusters}
df['area_ha'] = df['cluster'].map(area_dict)
df['costo_ha'] = df['costo_total'] / df['area_ha']

# 6.1) Área total por finca
area_finca = df.groupby('finca')['area_ha'].sum().reset_index(name='area_finca')

# 7) Métricas intermedias

# 7.1) Costo total mensual por finca y costo/ha mensual por finca
cost_monthly = df.groupby(['finca', 'mes'])['costo_total'].sum().reset_index()
cost_monthly = cost_monthly.merge(area_finca, on='finca')
cost_monthly['costo_ha_mes'] = cost_monthly['costo_total'] / cost_monthly['area_finca']

# 7.1b) Costo total mensual por actividad y costo/ha (usa área total)
total_area = df['area_ha'].sum()
cost_monthly_act = df.groupby(['actividad', 'mes'])['costo_total'].sum().reset_index()
cost_monthly_act['costo_ha_mes'] = cost_monthly_act['costo_total'] / total_area

# 7.2) Costo acumulado año calendario por finca
df['costo_ac_year'] = df.groupby(['finca', 'anio'])['costo_total'].cumsum()

# 7.3) Costo por hectárea total por lote
cost_ha_lote = df.groupby('cluster')['costo_total'].sum().reset_index()
cost_ha_lote['costo_ha_total'] = cost_ha_lote['costo_total'] / cost_ha_lote['cluster'].map(area_dict)

# 7.4) Costo total por actividad (y porcentaje)
act_cost = df.groupby('actividad')['costo_total'].sum().reset_index()
act_cost['pct'] = act_cost['costo_total'] / act_cost['costo_total'].sum()

# 7.5) Datos para Gantt (inicio y fin por lote y actividad)
gantt = (
    df.groupby(['cluster', 'actividad'])
      .agg(start=('fecha', 'min'), end=('fecha', 'max'))
      .reset_index()
)

# 7.6) Heatmap de costo/ha por actividad y lote
heat_cop_ha = df.groupby(['actividad', 'cluster'])['costo_ha'].sum().unstack(fill_value=0)

# Normalizar cada fila (actividad) para escalar colores
heat_norm = heat_cop_ha.copy()
for act in heat_norm.index:
    row = heat_norm.loc[act]
    mn, mx = row.min(), row.max()
    if mx > mn:
        heat_norm.loc[act] = (row - mn) / (mx - mn)

# 7.7) Clustering: kilos/ha vs costo/ha por lote
prod = df.groupby('cluster').agg(total_kilos=('kilos_cafe', 'sum')).reset_index()
prod['kilos_ha'] = prod['total_kilos'] / prod['cluster'].map(area_dict)
cost_cluster = cost_ha_lote[['cluster', 'costo_ha_total']]
cluster_df = prod.merge(cost_cluster, on='cluster').dropna()

scaler = MinMaxScaler()
features = scaler.fit_transform(cluster_df[['kilos_ha', 'costo_ha_total']])
kmeans = KMeans(n_clusters=3, random_state=42).fit(features)
cluster_df['cluster_label'] = kmeans.labels_

# Validación rápida (opcional)
print("► Costos mensuales por finca (muestra):")
display(cost_monthly.head())
print("\n► Costos mensuales por actividad (muestra):")
display(cost_monthly_act.head())
print("\n► Costo/ha por lote (muestra):")
display(cost_ha_lote.head())
print("\n► Datos Gantt (muestra):")
display(gantt.head())
print("\n► Heatmap input (muestra):")
display(heat_cop_ha.head())
print("\n► Clustering input (muestra):")
display(cluster_df.head())

# ------------------- Gráficas MVP Actualizado -------------------

# 1a) Costo/ha mensual por finca (columnas apiladas)
fig1 = px.bar(
    cost_monthly,
    x='mes',
    y='costo_ha_mes',
    color='finca',
    title='Costo/ha mensual por finca',
    labels={'costo_ha_mes': 'COP/ha', 'mes': 'Mes'}
)
fig1.show()

# 1b) Costo/ha mensual por actividad (columnas apiladas)
fig2 = px.bar(
    cost_monthly_act,
    x='mes',
    y='costo_ha_mes',
    color='actividad',
    title='Costo/ha mensual por actividad',
    labels={'costo_ha_mes': 'COP/ha', 'mes': 'Mes'}
)
fig2.show()

# 2) Cronograma de labores (Gantt)
fig3 = px.timeline(
    gantt,
    x_start='start',
    x_end='end',
    y='cluster',
    color='actividad',
    title='Cronograma de labores por lote'
)
fig3.update_yaxes(autorange='reversed')
fig3.show()

# 3) Heatmap de costo/ha por actividad y lote
#    - Valores en COP/ha (heat_cop_ha)
#    - Colores normalizados por actividad (heat_norm), pero mostramos texto con valores reales

fig4 = px.imshow(
    heat_norm.values,
    x=heat_norm.columns,
    y=heat_norm.index,
    aspect='auto',
    color_continuous_scale='Viridis',
    title='Heatmap – Costo/ha por actividad y lote (colores normalizados por actividad)',
    labels={'x': 'Lote', 'y': 'Actividad', 'color': 'Normalized'}
)
# Agregar los valores reales (COP/ha) como texto sobre cada celda
fig4.update_traces(
    text=heat_cop_ha.round(0).values,
    texttemplate='%{text}',
    textfont={'color': 'white'}
)
fig4.show()

# 4) Scatter clustering: kilos/ha vs costo/ha por lote
fig5 = px.scatter(
    cluster_df,
    x='kilos_ha',
    y='costo_ha_total',
    color='cluster_label',
    hover_data=['cluster'],
    title='Clustering de lotes: kilos/ha vs costo/ha'
)
fig5.show()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
► Costos mensuales por finca (muestra):


,finca,mes,costo_total,area_finca,costo_ha_mes
0,la_paloma,2024-01-01,19837000.0,3617.268542,5483.972165
1,la_paloma,2024-02-01,52038200.0,3617.268542,14386.048310
2,la_paloma,2024-03-01,11765100.0,3617.268542,3252.481773
3,la_paloma,2024-04-01,12709600.0,3617.268542,3513.590393
4,la_paloma,2024-05-01,17024600.0,3617.268542,4706.479434



► Costos mensuales por actividad (muestra):


,actividad,mes,costo_total,costo_ha_mes
0,actividad_general,2024-01-01,20673500.0,2055.849287
1,actividad_general,2024-02-01,12440300.0,1237.109434
2,actividad_general,2024-03-01,6437000.0,640.119083
3,actividad_general,2024-04-01,2433900.0,242.036016
4,actividad_general,2024-05-01,10517000.0,1045.849370



► Costo/ha por lote (muestra):


...

,cluster,actividad,start,end
0,la_paloma_0,control_maleza,2024-06-27,2024-06-27
1,la_paloma_0,fertilizacion,2024-01-25,2024-01-25
2,la_paloma_1,control_maleza,2024-02-02,2024-11-21
3,la_paloma_1,fertilizacion,2024-01-29,2024-07-12
4,la_paloma_10,control_maleza,2024-02-08,2024-03-27



► Heatmap input (muestra):


cluster,la_paloma_0,la_paloma_1,la_paloma_10,la_paloma_11,la_paloma_12,la_paloma_13,la_paloma_14,la_paloma_15,la_paloma_16,la_paloma_17,...,santa_rita_21,santa_rita_22,santa_rita_3,santa_rita_5,santa_rita_6,santa_rita_7,santa_rita_8,santa_rita_9,villa_libia_0,villa_libia_1
actividad,,,,,,,,,,,,,,,,,,,,,
control_maleza,37464.512898,312401.318503,125606.899563,338675.267608,31687.932292,187802.539115,44834.228725,694134.857107,2.300381e+04,68119.648027,...,11368.726286,24998.761065,108315.094510,55721.083113,63150.326762,9.305725e+04,29069.030327,51558.390161,8.791463e+03,56652.950049
control_roya,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,1.499019e+06,0.000000
fertilizacion,259526.898441,434558.544758,118837.465772,0.000000,0.000000,148613.027923,0.000000,231644.007693,1.707750e+06,0.000000,...,0.000000,0.000000,287110.324696,0.000000,0.000000,1.891937e+06,0.000000,0.000000,1.219480e+05,0.000000
renovacion,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,1.493146e+04,0.000000,0.000000,0.000000e+00,0.000000



► Clustering input (muestra):


,cluster,total_kilos,kilos_ha,costo_ha_total,cluster_label
0,la_paloma_0,0.0,0.0,296991.411339,1
1,la_paloma_1,0.0,0.0,746959.863261,1
2,la_paloma_10,0.0,0.0,244444.365334,1
3,la_paloma_11,0.0,0.0,338675.267608,1
4,la_paloma_12,0.0,0.0,31687.932292,1


In [ ]:
import re
import numpy as np
import pandas as pd
from IPython.display import display
import plotly.express as px
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

# Leer la hoja “Consolidado_agrochat” del Excel oficial
file_path = '/content/BASE DE DATOS OFICIAL.xlsx'
xls = pd.ExcelFile(file_path)
df_con = xls.parse('Consolidado_agrochat')

# Seleccionar columnas relevantes y crear identificador “cluster”
df_con = df_con[['id_finca', 'id_lote', 'fecha_actividad',
                 'nombre_actividad', 'cantidad_jornales', 'costo']].copy()
df_con = df_con.dropna(subset=['id_finca', 'id_lote'])
df_con['cluster'] = df_con['id_finca'].astype(int).astype(str) + "_" + df_con['id_lote'].astype(int).astype(str)

# Conversión de tipos y limpieza mínima
df_con['fecha_actividad'] = pd.to_datetime(df_con['fecha_actividad'], errors='coerce')
df_con['cantidad_jornales'] = pd.to_numeric(df_con['cantidad_jornales'], errors='coerce').fillna(0)
df_con['costo'] = pd.to_numeric(df_con['costo'], errors='coerce').fillna(0)

# Simular “área” por lote (5–20 ha)
np.random.seed(42)
unique_clusters = df_con['cluster'].unique()
area_dict = {cl: np.random.uniform(5, 20) for cl in unique_clusters}
df_con['area_ha'] = df_con['cluster'].map(area_dict)

# Agregar métricas por lote y actividad
agg_base = df_con.groupby('cluster').agg(
    jornales_total=('cantidad_jornales', 'sum'),
    cost_total=('costo', 'sum')
).reset_index()
agg_base['area_ha'] = agg_base['cluster'].map(area_dict)

# Simular kilos_total (entre 1000 y 5000 kg)
agg_base['kilos_total'] = np.random.uniform(1000, 5000, size=len(agg_base))

# Calcular métricas por hectárea
agg_base['kilos_ha'] = agg_base['kilos_total'] / agg_base['area_ha']
agg_base['cost_ha'] = agg_base['cost_total'] / agg_base['area_ha']

# Costo por actividad y lote
cost_act = df_con.groupby(['cluster', 'nombre_actividad'])['costo'].sum().reset_index()
cost_act['cost_ha_act'] = cost_act.apply(lambda r: r['costo'] / area_dict[r['cluster']], axis=1)

# Pivot para obtener columnas cost_ha_<actividad>
cost_act_wide = cost_act.pivot(index='cluster', columns='nombre_actividad', values='cost_ha_act').fillna(0)
cost_act_wide.columns = [f"cost_ha_{act}" for act in cost_act_wide.columns]

# Combinar variables en DataFrame para clustering
cluster_df = agg_base[['cluster', 'area_ha', 'kilos_total', 'kilos_ha', 'cost_total', 'cost_ha']].merge(
    cost_act_wide.reset_index(), on='cluster'
)

# Mostrar columnas usadas
display(cluster_df.head())

# Preparar features y escalar
features_cols = ['kilos_ha', 'cost_ha'] + [col for col in cluster_df.columns if col.startswith('cost_ha_')]
X = cluster_df[features_cols].values
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# KMeans clustering con k=3
kmeans = KMeans(n_clusters=3, random_state=42)
cluster_df['cluster_label'] = kmeans.fit_predict(X_scaled)

# Mostrar resultados
print("Datos con variables para clustering (muestra):")
display(cluster_df.head())

# Graficar kilos/ha vs cost_ha general coloreado por cluster
fig1 = px.scatter(
    cluster_df,
    x='kilos_ha',
    y='cost_ha',
    color='cluster_label',
    hover_data=['cluster'],
    title='Clustering de lotes: kilos/ha vs costo/ha (general)',
    labels={'kilos_ha':'Kilos por hectárea', 'cost_ha':'Costo por hectárea (COP)'}
)
fig1.show()

# Graficar kilos/ha vs cost_ha_fertilizacion si existe
if 'cost_ha_fertilizacion' in cluster_df.columns:
    fig2 = px.scatter(
        cluster_df,
        x='kilos_ha',
        y='cost_ha_fertilizacion',
        color='cluster_label',
        hover_data=['cluster'],
        title='Kilos/ha vs Costo/ha Fertilización',
        labels={'kilos_ha':'Kilos por hectárea', 'cost_ha_fertilizacion':'Costo/ha Fertilización (COP)'}
    )
    fig2.show()

# Graficar cost_ha_control_maleza vs cost_ha_control_roya si existen
if 'cost_ha_control_maleza' in cluster_df.columns and 'cost_ha_control_roya' in cluster_df.columns:
    fig3 = px.scatter(
        cluster_df,
        x='cost_ha_control_maleza',
        y='cost_ha_control_roya',
        color='cluster_label',
        hover_data=['cluster'],
        title='Costo/ha Control Maleza vs Control Roya',
        labels={'cost_ha_control_maleza':'Costo/ha Control Maleza', 'cost_ha_control_roya':'Costo/ha Control Roya'}
    )
    fig3.show()


,cluster,area_ha,kilos_total,kilos_ha,cost_total,cost_ha,cost_ha_Amarre,cost_ha_Cirugía,cost_ha_Control Platano,cost_ha_Control de Corona,...,cost_ha_Repique,cost_ha_Reporte de siembras,cost_ha_Reporte de siniestros,cost_ha_Resiembra,cost_ha_Saneamiento,cost_ha_Saneamiento plantacion racimos agobiados,cost_ha_Subsolado,cost_ha_aplicacion_agroquimicos,cost_ha_control de sigatoka,cost_ha_control de speckling
0,40301_1,10.618102,4962.020568,467.317103,15000.00000,1412.681881,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,40301_2,19.260715,2650.470708,137.610196,15000.00000,778.787304,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,40301_3,15.979909,2488.072343,155.700031,15000.00000,938.678680,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,40301_4,13.979877,4105.651843,293.682968,15000.00000,1072.970794,0.000000,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,40321_1,7.340280,2363.214161,321.951518,613142.30772,83531.192349,7316.531181,0.0,0.0,0.0,...,3353.466889,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Datos con variables para clustering (muestra):


,cluster,area_ha,kilos_total,kilos_ha,cost_total,cost_ha,cost_ha_Amarre,cost_ha_Cirugía,cost_ha_Control Platano,cost_ha_Control de Corona,...,cost_ha_Reporte de siembras,cost_ha_Reporte de siniestros,cost_ha_Resiembra,cost_ha_Saneamiento,cost_ha_Saneamiento plantacion racimos agobiados,cost_ha_Subsolado,cost_ha_aplicacion_agroquimicos,cost_ha_control de sigatoka,cost_ha_control de speckling,cluster_label
0,40301_1,10.618102,4962.020568,467.317103,15000.00000,1412.681881,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,40301_2,19.260715,2650.470708,137.610196,15000.00000,778.787304,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
2,40301_3,15.979909,2488.072343,155.700031,15000.00000,938.678680,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
3,40301_4,13.979877,4105.651843,293.682968,15000.00000,1072.970794,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
4,40321_1,7.340280,2363.214161,321.951518,613142.30772,83531.192349,7316.531181,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
